In [1]:
from data.data_utils import get_distillation_dataloader

In [2]:
from configs import dataset as DATA_CONFIG
from configs import fsdp_config as FSDP_CONFIG
from configs import train_config as TRAIN_CONFIG
from configs import distillation_config as DISTIL_CONFIG

In [3]:
train_config, fsdp_config, distil_config, data_config = TRAIN_CONFIG(), FSDP_CONFIG(), DISTIL_CONFIG(), DATA_CONFIG()


In [4]:
train_config

train_config(project_name=None, model_name='meta-llama/Llama-2-7b-hf', enable_fsdp=False, low_cpu_fsdp=False, run_validation=True, batch_size_training=8, batching_strategy='padding', context_length=None, gradient_accumulation_steps=1, num_epochs=1, num_workers_dataloader=2, lr=1e-06, weight_decay=0.1, seed=42, use_fp16=False, mixed_precision=True, val_batch_size=1, peft_method='lora', use_peft=False, output_dir='', freeze_layers=False, num_freeze_layers=1, quantization=False, save_model=True, save_step=1000, save_optimizer=False, use_fast_kernels=False, distillation=False, save_all=False, training_size=1, encoder_decoder=False)

In [5]:
args_dict = {'model_name': 'EleutherAI/pythia-410m-deduped', 'dataset.file': 'datasets-m/loader/squad.py', 'lr': 1e-06, 'num_epochs': 5, 'batch_size_training': 1, 'val_batch_size': 1, 'output_dir': 'train/output/path', 'distillation_config.model_name': 'mistralai/Mistral-7B-Instruct-v0.2', 'distillation': True, 'distillation_config.enable_fsdp': False, 'distillation_config.pure_bf16': True, 'distillation_config.distil_factor': 1.5, 'save_step': 100, }

In [7]:
def update_config(config, update_dict, isSubmodule=False):
    if isinstance(config, (tuple, list)):
        for c in config:
            update_config(c, update_dict, isSubmodule)
    else:
        for k, v in update_dict.items():
            if "." in k:
                config_name, param_name = k.split(".", 1)
                if type(config).__name__ == config_name:
                    if hasattr(config, param_name):
                        setattr(config, param_name, v)
            elif not isSubmodule and hasattr(config, k):
                setattr(config, k, v)


In [8]:
update_config((train_config, fsdp_config, data_config), args_dict)
update_config((distil_config), args_dict, isSubmodule=True,)

In [9]:
train_config

train_config(project_name=None, model_name='EleutherAI/pythia-410m-deduped', enable_fsdp=False, low_cpu_fsdp=False, run_validation=True, batch_size_training=1, batching_strategy='padding', context_length=None, gradient_accumulation_steps=1, num_epochs=5, num_workers_dataloader=2, lr=1e-06, weight_decay=0.1, seed=42, use_fp16=False, mixed_precision=True, val_batch_size=1, peft_method='lora', use_peft=False, output_dir='train/output/path', freeze_layers=False, num_freeze_layers=1, quantization=False, save_model=True, save_step=100, save_optimizer=False, use_fast_kernels=False, distillation=True, save_all=False, training_size=1, encoder_decoder=False)

In [10]:
from models.models_utils import get_distillation_models

/home/brimmann/works/llm-recipes/models/checkpoint_handler.py:7: DeprecationWarning: `torch.distributed._shard.checkpoint` will be deprecated, use `torch.distributed.checkpoint` instead
  import torch.distributed._shard.checkpoint as dist_cp
/home/brimmann/works/llm-recipes/models/tools.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
student_tokenizer, teacher_tokenizer, model = get_distillation_models(train_config, distil_config, fsdp_config, 0, args_dict)

The `GPTNeoXSdpaAttention` class is deprecated in favor of simply modifying the `config._attn_implementation`attribute of the `GPTNeoXAttention` class! It will be removed in v4.48


--> Model EleutherAI/pythia-410m-deduped

--> EleutherAI/pythia-410m-deduped has 405.334016 Million params



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

--> Model mistralai/Mistral-7B-Instruct-v0.2

--> mistralai/Mistral-7B-Instruct-v0.2 has 7241.732096 Million params



In [10]:
train_dataloader, teacher_train_dataloader, eval_dataloader, teacher_eval_dataloader = get_distillation_dataloader(data_config, train_config, distil_config, student_tokenizer, teacher_tokenizer, 0)

train_dl_kwargs:  {'batch_sampler': <data.sampler.LengthBasedBatchSampler object at 0x746821432b80>, 'collate_fn': DataCollatorForSeq2Seq(tokenizer=GPTNeoXTokenizerFast(name_or_path='EleutherAI/pythia-410m-deduped', vocab_size=50254, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<|padding|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50254: AddedToken("                        ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50255: AddedToken("                       ", rstrip=False, lstrip=False, single_word=False, normalized=True, 

In [11]:
student_el1 = next(iter(train_dataloader))
teacher_ele1 = next(iter(teacher_train_dataloader))

In [12]:
teacher_tokenizer.decode(teacher_ele1["input_ids"][0])

'<s> [INST] You are an agent answering questions as part of a reading comprehension activity. You must read and understand the context text step by step. Answers consist of 0 to 5 words, exclusively continuous words taken from the contextual text provided.\n\nTitle: Christine\'s boyfriend\nContext: Patrick Harris (Tim DeKay), Old Christine\'s new boyfriend, who she meets in a video store and starts dating.\nQuestion: Who played patrick on new adventures of old christine? [/INST] Answer: Tim DeKay</s> [INST] Title: June 14, 2018: Death Row Inmates\nContext: As of June 14, 2018, there were 2,718 death row inmates in the United States.\nQuestion: Total number of death row inmates in the us? [/INST] Answer: 2,718</s> [INST] Title: Modern Communism\nContext: Most modern forms of communism are grounded at least nominally in Marxism, an ideology conceived by noted sociologist Karl Marx during the mid nineteenth century.\nQuestion: Who came up with the idea of communism ? [/INST] Answer: Karl 

In [13]:
student_tokenizer.decode(student_el1["input_ids"][0])

'<|endoftext|>Context: Time has long been a major subject of study in religion, philosophy, and science, but defining it in a manner applicable to all fields without circularity has consistently eluded scholars. Nevertheless, diverse fields such as business, industry, sports, the sciences, and the performing arts all incorporate some notion of time into their respective measuring systems. Some simple definitions of time include "time is what clocks measure", which is a problematically vague and self-referential definition that utilizes the device used to measure the subject as the definition of the subject, and "time is what keeps everything from happening at once", which is without substantive meaning in the absence of the definition of simultaneity in the context of the limitations of human sensation, observation of events, and the perception of such events.\nQuestion: Fields such as business, industry, sports, science, and performing arts incorporate some notion of what into their m

In [ ]:
!uv run python -m datasets-m.generator --model_id "mistralai/Mistral-7B-Instruct-v0.2" --dataset_id "GEM/squad_v2" --num_workers 1 --bfloat --batch_size 1

In [ ]:
from datasets import load_from_disk
d = load_from_disk("/home/brimmann/works/llm-recipes/datasets/hf/Mistral-7B-Instruct-v0.2-squad")

In [ ]:
from datasets import DatasetDict

In [ ]:
d = DatasetDict({"train": d})

In [ ]:
d

In [ ]:
d.save_to_disk("/home/brimmann/works/llm-recipes/datasets/hf/Mistral-7B-Instruct-v0.2-squad")